### STATE_DATA_Run

Prepares BATCH code to run STATE_DATA project data and analysis.

Batch code using BSUB is submitted through BASH terminal


In [ ]:
#Here is code to extract the most recent code updates
# ZIP file downloaded (print the name)
#Extract contents of main_folder_name to the ~ home directory.

import os
import sys
from glob import glob

def find_most_recent_zip():
    # List all ZIP files in the current directory
    zip_files = glob('*.zip')
    # Find the most recent ZIP file based on modification time
    if not zip_files:
        return None
    latest_zip = max(zip_files, key=os.path.getmtime)
    print(latest_zip)
    return latest_zip

def generate_unzip_command(folder_name):
    zip_file = find_most_recent_zip()
    if zip_file is None:
        print("No ZIP files found in the current directory.")
        sys.exit(1)
    # Generate the command
    command = f"unzip {zip_file} '{folder_name}/*' -d ~/ && mv ~/{folder_name}/* ~/ && rmdir ~/{folder_name}"
    return command

if __name__ == "__main__":
    if len(sys.argv) != 2:
        print("Usage: python generate_unzip_command.py <folder_name_inside_zip>")
        sys.exit(1)
    folder_name = sys.argv[1]
    command = generate_unzip_command(folder_name)
    print(command)


In [4]:
#Import code generation functions
from cpq_tools import write_bsub

In [5]:
def _project_bsub(project, verbose = False):
    """Generate BATCH job scheduler .sh files and bsub call for running each part of the project"""
    #Define dictionary of project files
    project_files = {'processing':'STATE_DATA_process.py',
                'analysis':'STATE_DATA_analysis.py'}
    code_file = 'out_dir/' + project_files[project]
    bsub_data = write_bsub(project, code_file)
    if verbose:
        print(bsub_data['bsub_code'])
    return(bsub_data['out_command'])
    

In [6]:
_project_bsub('processing', verbose=True)

#!/bin/bash
#BSUB -J processing # LSF Job Name
#BSUB -o job_output.%J # Name of the job output file
#BSUB -e job_error.%J # Name of the job error file
module unload python
module load python/3.8
python --version
python $HOME/out_dir/STATE_DATA_process.py



'BSUB < processing.sh'

In [4]:
_project_bsub('analysis')

'BSUB < analysis.sh'